In [ ]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import json
import logging

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Acceder a la variable de entorno PYTHON_URL_BACK
PYTHON_URL_BACK = os.getenv("PYTHON_URL_BACK")

# Configurar los logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def obtener_historial_ventas(correo_cliente):
    """
    Obtiene el historial de ventas de un cliente y retorna los datos en formato JSON.
    """
    # Obtener todos los usuarios para encontrar el ID del cliente
    response = requests.get(f"{PYTHON_URL_BACK}/user")
    if response.status_code != 200:
        logging.error("Error al obtener la lista de usuarios.")
        return json.dumps({"error": "Error al obtener la lista de usuarios."})
    
    # Convertir la respuesta en un DataFrame y buscar el usuario por correo
    users = response.json()
    user_df = pd.DataFrame(users)
    
    cliente = user_df[user_df['email'] == correo_cliente]
    
    if cliente.empty:
        logging.warning(f"No se ha encontrado un usuario con el correo: {correo_cliente}")
        return json.dumps({"error": f"No se ha encontrado un usuario con el correo: {correo_cliente}"})
    
    # Obtener el nombre completo del cliente combinando 'firstName' y 'lastName'
    if 'firstName' in cliente.columns and 'lastName' in cliente.columns:
        full_name = f"{cliente['firstName'].values[0]} {cliente['lastName'].values[0]}"
    else:
        logging.warning("No se ha encontrado el nombre completo del cliente.")
        return json.dumps({"error": "No se ha encontrado el nombre completo del cliente."})
    
    user_id = cliente['id'].values[0]
    
    # Obtener el historial de ventas del cliente usando su ID
    response = requests.get(f"{PYTHON_URL_BACK}/ordersUser/{user_id}")
    if response.status_code != 200:
        logging.error("Error al obtener el historial de ventas del cliente.")
        return json.dumps({"error": "Error al obtener el historial de ventas del cliente."})
    
    # Convertir las ventas a DataFrame
    ventas = response.json()
    
    if not ventas:
        logging.warning(f"El usuario {full_name} no ha realizado compras.")
        return json.dumps({"message": f"El usuario {full_name} no ha realizado compras."})
    
    # Crear una lista de descripciones de productos para cada venta
    data = []
    for venta in ventas:
        for producto in venta['orderLines']:
            data.append({
                'ID': venta['id'],
                'Clients': f"{producto['name']} (ID Venda: {venta['id']})",
                'Ventas': venta['productCount'],
                'Diners': float(venta['total']),
                'Diners/Venda': float(venta['total']) / venta['productCount'],
            })
    
    # Crear un DataFrame con la información extraída
    df_ventas = pd.DataFrame(data)
    
    # Convertir el DataFrame a un diccionario y añadir el nombre del cliente
    resultado = {
        "nombre_cliente": full_name,
        "ventas": df_ventas.to_dict('records')
    }
    
    return json.dumps(resultado)

if __name__ == "__main__":
    import sys
    if len(sys.argv) > 1:
        correo_cliente = sys.argv[1]
        print(obtener_historial_ventas(correo_cliente))
    else:
        print(json.dumps({"error": "No se proporcionó un correo electrónico."}))

KeyboardInterrupt: 